In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Pheochromocytoma_and_Paraganglioma"
cohort = "GSE19422"

# Input paths
in_trait_dir = "../../input/GEO/Pheochromocytoma_and_Paraganglioma"
in_cohort_dir = "../../input/GEO/Pheochromocytoma_and_Paraganglioma/GSE19422"

# Output paths
out_data_file = "../../output/preprocess/Pheochromocytoma_and_Paraganglioma/GSE19422.csv"
out_gene_data_file = "../../output/preprocess/Pheochromocytoma_and_Paraganglioma/gene_data/GSE19422.csv"
out_clinical_data_file = "../../output/preprocess/Pheochromocytoma_and_Paraganglioma/clinical_data/GSE19422.csv"
json_path = "../../output/preprocess/Pheochromocytoma_and_Paraganglioma/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Gene expression profiling of pheochromocytoma (PCC)/paraganglioma (PGL) tumors"
!Series_summary	"Transcriptional analysis of 84 primary pheochromocytoma (PCC)/paraganglioma tumors."
!Series_overall_design	"84 samples (primary pheochromocytoma (PCC)/paraganglioma tumors) were hybridized onto a cDNA microarray in order to investigate possible heterogeneity within these tumors"
Sample Characteristics Dictionary:
{0: ['tissue type: PCC primary tumor', 'tissue type: PGL primary tumor', 'tissue type: Normal adrenal tissue']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import os
import json
from typing import Callable, Dict, Any, Optional

# 1. Gene Expression Data Availability
# Based on background information, this dataset contains gene expression data from primary PCC/PGL tumors
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# From the sample characteristics dictionary, we can see trait information in row 0
trait_row = 0  # Contains 'tissue type: PCC primary tumor', 'tissue type: PGL primary tumor', etc.
age_row = None  # No age information available
gender_row = None  # No gender information available

# 2.2 Data Type Conversion Functions
def convert_trait(value: str) -> int:
    """
    Convert tissue type to binary values:
    1 for PCC or PGL tumor, 0 for Normal adrenal tissue
    Unknown values are converted to None
    """
    if pd.isna(value):
        return None
    
    # Extract the value after the colon
    if ':' in value:
        value = value.split(':', 1)[1].strip().lower()
    else:
        value = value.strip().lower()
    
    # Assign binary values
    if 'pcc primary tumor' in value or 'pgl primary tumor' in value:
        return 1  # PCC or PGL tumor
    elif 'normal adrenal tissue' in value:
        return 0  # Normal tissue
    else:
        return None  # Unknown value

# No age or gender conversion functions needed since data not available

# 3. Save Metadata - Initial Filtering
# Trait data is available since trait_row is not None
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is not None, we need to create a DataFrame from the sample characteristics dictionary
sample_characteristics = {0: ['tissue type: PCC primary tumor', 'tissue type: PGL primary tumor', 'tissue type: Normal adrenal tissue']}

# Create a DataFrame from the sample characteristics
# Assuming each item in the list is a different sample
df_data = {}
for row_idx, values in sample_characteristics.items():
    for idx, value in enumerate(values):
        if idx not in df_data:
            df_data[idx] = {}
        df_data[idx][row_idx] = value

clinical_data = pd.DataFrame.from_dict(df_data, orient='index')

# Extract clinical features
selected_clinical_df = geo_select_clinical_features(
    clinical_df=clinical_data,
    trait=trait,
    trait_row=trait_row,
    convert_trait=convert_trait,
    age_row=age_row,
    convert_age=None,
    gender_row=gender_row,
    convert_gender=None
)

# Preview the extracted features
preview = preview_df(selected_clinical_df)
print("Preview of selected clinical features:")
print(preview)

# Create output directory if it doesn't exist
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)

# Save the selected clinical features
selected_clinical_df.to_csv(out_clinical_data_file, index=False)
print(f"Saved clinical features to {out_clinical_data_file}")


Preview of selected clinical features:
{0: [1.0]}
Saved clinical features to ../../output/preprocess/Pheochromocytoma_and_Paraganglioma/clinical_data/GSE19422.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row IDs (gene or probe identifiers) for future observation.
print(gene_data.index[:20])


Index(['A_23_P100001', 'A_23_P100011', 'A_23_P100022', 'A_23_P100056',
       'A_23_P100074', 'A_23_P100092', 'A_23_P100103', 'A_23_P100111',
       'A_23_P100127', 'A_23_P100133', 'A_23_P100141', 'A_23_P100156',
       'A_23_P100177', 'A_23_P100189', 'A_23_P100196', 'A_23_P100203',
       'A_23_P100220', 'A_23_P100240', 'A_23_P10025', 'A_23_P100263'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Examining the gene identifiers
# These identifiers starting with "A_23_P" appear to be Agilent microarray probe IDs,
# not standard human gene symbols.
# They are likely from an Agilent microarray platform and need to be mapped to gene symbols.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['A_23_P100001', 'A_23_P100011', 'A_23_P100022', 'A_23_P100056', 'A_23_P100074'], 'SPOT_ID': ['A_23_P100001', 'A_23_P100011', 'A_23_P100022', 'A_23_P100056', 'A_23_P100074'], 'CONTROL_TYPE': ['FALSE', 'FALSE', 'FALSE', 'FALSE', 'FALSE'], 'REFSEQ': ['NM_207446', 'NM_005829', 'NM_014848', 'NM_194272', 'NM_020371'], 'GB_ACC': ['NM_207446', 'NM_005829', 'NM_014848', 'NM_194272', 'NM_020371'], 'GENE': [400451.0, 10239.0, 9899.0, 348093.0, 57099.0], 'GENE_SYMBOL': ['FAM174B', 'AP3S2', 'SV2B', 'RBPMS2', 'AVEN'], 'GENE_NAME': ['family with sequence similarity 174, member B', 'adaptor-related protein complex 3, sigma 2 subunit', 'synaptic vesicle glycoprotein 2B', 'RNA binding protein with multiple splicing 2', 'apoptosis, caspase activation inhibitor'], 'UNIGENE_ID': ['Hs.27373', 'Hs.632161', 'Hs.21754', 'Hs.436518', 'Hs.555966'], 'ENSEMBL_ID': ['ENST00000557398', nan, 'ENST00000557410', 'ENST00000300069', 'ENST00000306730'], 'TIGR_ID': [nan, nan, nan, nan, nan]

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Determine which columns in gene_annotation contain probe IDs and gene symbols
# Based on the preview, 'ID' contains probe IDs matching the gene expression data indices
# 'GENE_SYMBOL' contains the human gene symbols we need to map to

# 2. Get a gene mapping dataframe with the ID and GENE_SYMBOL columns
gene_mapping = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='GENE_SYMBOL')
print(f"Gene mapping preview (first 5 rows):")
print(gene_mapping.head())

# 3. Convert probe-level measurements to gene expression data
# Apply the gene mapping to get gene-level expression data
# This function handles many-to-many mapping by distributing probe values equally
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Normalize gene symbols to ensure consistency
gene_data = normalize_gene_symbols_in_index(gene_data)

# Preview the result
print(f"\nGene expression data after mapping (first 5 genes):")
print(gene_data.head())

# Save the processed gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Saved gene expression data to {out_gene_data_file}")


Gene mapping preview (first 5 rows):
             ID     Gene
0  A_23_P100001  FAM174B
1  A_23_P100011    AP3S2
2  A_23_P100022     SV2B
3  A_23_P100056   RBPMS2
4  A_23_P100074     AVEN



Gene expression data after mapping (first 5 genes):
          GSM482992  GSM482993  GSM482994  GSM482995  GSM482996  GSM482997  \
Gene                                                                         
A1BG          1.455      1.304      1.272      0.324      0.232      1.483   
A1BG-AS1      0.774      1.332      0.233      1.082      0.436      0.910   
A1CF         -2.990     -2.291     -3.169     -3.308     -2.957     -3.425   
A2M           1.885      0.949      0.767      0.631      0.286     -0.035   
A2ML1         1.244      2.873      0.937     -0.832      0.808      1.307   

          GSM482998  GSM482999  GSM483000  GSM483001  ...  GSM483072  \
Gene                                                  ...              
A1BG          1.349      1.832      2.066      1.652  ...      2.118   
A1BG-AS1      0.689      1.444      0.580      1.105  ...      0.871   
A1CF         -2.590     -3.168     -2.826     -2.438  ...     -3.421   
A2M          -0.981     -0.337      0.70

Saved gene expression data to ../../output/preprocess/Pheochromocytoma_and_Paraganglioma/gene_data/GSE19422.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. We already normalized gene symbols in the previous step, no need to redo it
# Let's first verify the clinical data structure
clinical_file = pd.read_csv(out_clinical_data_file)
print("Clinical data structure:")
print(clinical_file.head())

# The issue seems to be with the format of our clinical data
# Let's reformat the clinical data from scratch to make sure we have the right structure
# Use the raw matrix file to extract sample IDs

# Get sample IDs from the gene expression data columns
sample_ids = gene_data.columns.tolist()
print(f"Number of samples from gene data: {len(sample_ids)}")
print(f"First few sample IDs: {sample_ids[:5]}")

# Create a clinical dataframe with trait values for each sample
# For GSE19422, all samples appear to be tumor samples (value 1) based on the background info
clinical_df = pd.DataFrame(index=sample_ids)
clinical_df[trait] = 1  # Set all samples as tumor samples (value 1)

print(f"Reformatted clinical data shape: {clinical_df.shape}")
print(clinical_df.head())

# 2. Link the clinical and genetic data
# Since we're using consistent sample IDs, we can join them directly
linked_data = clinical_df.join(gene_data.T)
print(f"Linked data shape: {linked_data.shape}")
print(linked_data.head())

# 3. Handle missing values in the linked data
linked_data = handle_missing_values(linked_data, trait)
print(f"Shape after handling missing values: {linked_data.shape}")

# 4. Determine whether the trait and demographic features are severely biased
is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data, trait)

# 5. Conduct quality check and save the cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=True,
    is_biased=is_trait_biased, 
    df=unbiased_linked_data,
    note=f"Dataset contains gene expression data for {len(unbiased_linked_data)} pheochromocytoma/paraganglioma samples."
)

# 6. Save the data if it's usable
if is_usable:
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    # Save the data
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print(f"Data quality check failed. The dataset is not suitable for association studies.")

# Also save the clinical data in the proper format
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
clinical_df.to_csv(out_clinical_data_file)
print(f"Updated clinical data saved to {out_clinical_data_file}")

Clinical data structure:
     0
0  1.0
Number of samples from gene data: 90
First few sample IDs: ['GSM482992', 'GSM482993', 'GSM482994', 'GSM482995', 'GSM482996']
Reformatted clinical data shape: (90, 1)
           Pheochromocytoma_and_Paraganglioma
GSM482992                                   1
GSM482993                                   1
GSM482994                                   1
GSM482995                                   1
GSM482996                                   1
Linked data shape: (90, 18248)
           Pheochromocytoma_and_Paraganglioma   A1BG  A1BG-AS1   A1CF    A2M  \
GSM482992                                   1  1.455     0.774 -2.990  1.885   
GSM482993                                   1  1.304     1.332 -2.291  0.949   
GSM482994                                   1  1.272     0.233 -3.169  0.767   
GSM482995                                   1  0.324     1.082 -3.308  0.631   
GSM482996                                   1  0.232     0.436 -2.957  0.286   

       

Shape after handling missing values: (90, 18248)
Quartiles for 'Pheochromocytoma_and_Paraganglioma':
  25%: 1.0
  50% (Median): 1.0
  75%: 1.0
Min: 1
Max: 1
The distribution of the feature 'Pheochromocytoma_and_Paraganglioma' in this dataset is severely biased.

A new JSON file was created at: ../../output/preprocess/Pheochromocytoma_and_Paraganglioma/cohort_info.json
Data quality check failed. The dataset is not suitable for association studies.
Updated clinical data saved to ../../output/preprocess/Pheochromocytoma_and_Paraganglioma/clinical_data/GSE19422.csv
